# **Extração Simples de Comentários da App Store e Play Store**
Esse notebook trás uma forma simples de coleta de dados das principais lojas de aplicativos utilizando uma biblioteca de scrapp para dados da Google Play e uma API pública da Apple.

| Aplicativo | Play Store ID           | App Store ID |
| ---------- | ----------------------- | ------------ | 
| Claro      | com.nvt.cs              | 526829675    |
| TIM        | br.com.timbrasil.meutim | 668591218    |
| Vivo       | br.com.vivo             | 475915064    |



In [27]:
import requests
import time
import pandas as pd
from google_play_scraper import Sort, reviews

**Play Store**

In [28]:
qtd_comentarios = 50000

dados_android = []

apps = {
    "Claro": "com.nvt.cs",
    "TIM": "br.com.timbrasil.meutim",
    "Vivo": "br.com.vivo"
}

for app, app_id in apps.items():
    result, _ = reviews(
        app_id,
        lang='pt',
        country='br',
        sort=Sort.NEWEST,
        count=qtd_comentarios
    )
        
    for review in result:
        dados_android.append({
            "app": app,
            "system": "Android",
            "score": review.get("score", None),
            "content": review.get("content", None),
            "app_version": review.get("appVersion", None),
            "date": review.get("at", None),
            "reply_content": review.get("replyContent", None),
            "reply_date": review.get("repliedAt", None)
        })

comentarios_android = pd.DataFrame(dados_android, columns=["app", "system", "score", "content",
                                                           "app_version", "date", "reply_content", "reply_date"])

comentarios_android.head()

,app,system,score,content,app_version,date,reply_content,reply_date
0,Claro,Android,5,legal 😎,18.7.0,2025-05-27 18:54:37,None,NaT
1,Claro,Android,5,Encontro facilidade por aqui 👏,18.6.0,2025-05-27 18:10:04,None,NaT
2,Claro,Android,5,muito bom,18.7.0,2025-05-27 17:49:18,None,NaT
3,Claro,Android,1,ruim que vc deseja não tem informações,None,2025-05-27 17:46:20,"Olá,Junior! Estou aqui para te ajudar. Por gen...",2025-05-28 08:58:31
4,Claro,Android,5,melhor do mundo,18.7.0,2025-05-27 17:20:22,None,NaT


**Dados App Store** \
Observação: A API pública permite coletar no máxinmo 10 páginas de comentários

In [29]:
page = 100
dados_ios = []

apps = {
    "Claro": "526829675",
    "TIM": "668591218",
    "Vivo": "475915064"
}

for app, app_id in apps.items():
    for p in range(1, page + 1):  # Coletar da página 1 até o limite máximo
        url = f'https://itunes.apple.com/rss/customerreviews/page={p}/id={app_id}/sortby=mostrecent/json'
        
        try:
            response = requests.get(url)
            data = response.json()
        except Exception as e:
            print(f"Erro ao acessar página {p} do app {app}: {e}")
            break

        # Se não houver mais comentários, para o loop
        if "feed" not in data or "entry" not in data["feed"]:
            print(f"Sem mais comentários na página {p} para o app {app}.")
            break

        for entry in data["feed"]["entry"][1:]:  # Ignora o primeiro, que são infos do app
            dados_ios.append({
                "app": app,
                "system": "iOS",
                "score": entry['im:rating']['label'],
                "content": entry['content']['label'],
                "app_version": entry['im:version']['label']
            })

        if p % 10 == 0:
            print(f"Pausa após coletar {p} páginas do app {app}...")
            time.sleep(5)

comentarios_ios = pd.DataFrame(dados_ios, columns=["app", "system", "score", "content","app_version"])

comentarios_ios.head()

Sem mais comentários na página 5 para o app Claro.
Sem mais comentários na página 6 para o app TIM.
Pausa após coletar 10 páginas do app Vivo...
Erro ao acessar página 11 do app Vivo: Expecting value: line 1 column 1 (char 0)


,app,system,score,content,app_version
0,Claro,iOS,1,Tenho essa aplicativo instalado há anos e quan...,17.53.0
1,Claro,iOS,1,"Poluído, muito ruim, esconde informação",17.48.0
2,Claro,iOS,1,App is not at all intuitive. One thing it does...,17.46.0
3,Claro,iOS,4,"Boa tarde, sou cliente Claro e não vou mudar, ...",17.27.0
4,Claro,iOS,1,Ate hoje nao ha suporte para eSIM… tudo relaci...,17.26.0


In [30]:
#Juntando as bases
comentarios = pd.concat([comentarios_android, comentarios_ios], ignore_index=True)
comentarios_android.shape, comentarios_ios.shape, comentarios.shape

((150000, 8), (898, 5), (150898, 8))

In [31]:
comentarios.to_csv("comentarios.csv", index=False)